<h2>Exemplo usando o Apache Solr com o R pelo jupyter Notebook. <h2>

De: https://cran.r-project.org/web/packages/solrium/solrium.pdf
<br/>
https://cran.r-project.org/web/packages/solrium/solrium.pdf
<br/>
https://cran.r-project.org/web/packages/jsonlite/jsonlite.pdf
<br/>


In [12]:
library("solrium")

In [2]:
(cli <- SolrClient$new(host = "api.plos.org", path = "search", port = NULL, errors = "complete"))

<Solr Client>
  host: api.plos.org
  path: search
  port: 
  scheme: http
  errors: complete
  proxy:

In [3]:
cli$search(params = list(q='*:*', rows=2, fl='id'))

id
10.1371/journal.pone.0058099/materials_and_methods
10.1371/journal.pone.0030394/introduction


In [4]:
cli$group(params = list(q='*:*', group.field='journal', rows=5, group.limit=1,
                        group.sort='publication_date desc',
                        fl='publication_date, score'))

groupValue,numFound,start,publication_date,score
plos one,1800398,0,2019-03-27T00:00:00Z,1
none,66476,0,2012-10-23T00:00:00Z,1
plos genetics,67060,0,2019-03-27T00:00:00Z,1
plos biology,37280,0,2019-03-27T00:00:00Z,1
plos pathogens,60083,0,2019-03-26T00:00:00Z,1


In [5]:
cli$group(params = list(q = '*:*', group.field = 'journal', group.limit = 1,
                        group.sort = 'publication_date asc',
                        fl = c('publication_date', 'score'),
                        fq = "publication_date:[1900-01-01T00:00:00Z TO *]"))

groupValue,numFound,start,publication_date,score
plos one,1800398,0,2006-12-20T00:00:00Z,1
none,57532,0,2005-08-23T00:00:00Z,1
plos genetics,67060,0,2005-06-17T00:00:00Z,1
plos biology,37280,0,2003-08-18T00:00:00Z,1
plos pathogens,60083,0,2005-07-22T00:00:00Z,1
plos computational biology,52792,0,2005-06-24T00:00:00Z,1
plos medicine,26406,0,2004-09-07T00:00:00Z,1
plos neglected tropical diseases,57268,0,2007-08-30T00:00:00Z,1
plos clinical trials,521,0,2006-04-21T00:00:00Z,1
plos medicin,9,0,2012-04-17T00:00:00Z,1


In [6]:
gq <- 'publication_date:[2013-01-01T00:00:00Z TO 2013-12-31T00:00:00Z]'
cli$group(
  params = list(q='*:*', group.query = gq,
                group.limit = 3, group.sort = 'publication_date desc',
                fl = 'publication_date'))

numFound,start,publication_date
307076,0,2013-12-31T00:00:00Z
307076,0,2013-12-31T00:00:00Z
307076,0,2013-12-31T00:00:00Z


In [7]:
cli$group(params = list(q='*:*', group.field='journal', rows=5,
                        group.limit=3, group.sort='publication_date desc',
                        group.format='simple', fl='journal, publication_date'))

numFound,start,journal,publication_date
2168293,0,PLOS ONE,2019-03-27T00:00:00Z
2168293,0,PLOS ONE,2019-03-27T00:00:00Z
2168293,0,PLOS ONE,2019-03-27T00:00:00Z
2168293,0,NA,2012-10-23T00:00:00Z
2168293,0,NA,2012-10-23T00:00:00Z


In [8]:
cli$facet(params = list(q='*:*', facet.field='journal', facet.query=c('cell', 'bird')))

term,value
cell,175285
bird,18639
term,value
plos one,1800398
plos genetics,67060
plos pathogens,60083
plos neglected tropical diseases,57268
plos computational biology,52792
plos biology,37280
plos medicine,26406


In [9]:
cli$highlight(params = list(q='alcohol', hl.fl = 'abstract', rows=2))

names,abstract
10.1371/journal.pone.0201042,Acute <em>alcohol</em> administration can lead to a loss of control over drinking. Several models argue
10.1371/journal.pone.0185457,Objectives: <em>Alcohol</em>-related morbidity and mortality are significant public health issues


In [10]:
out <- cli$stats(params = list(q='ecology', stats.field=c('counter_total_all','alm_twitterCount'), stats.facet='journal'))

In [11]:
out

,min,max,count,missing,sum,sumOfSquares,mean,stddev
counter_total_all,0,1322780,47226,0,263688870,1.119526e+13,5583.552916,14348.70574
alm_twitterCount,0,3438,47226,0,304235,8.148536e+07,6.442108,41.03619
journal,min,max,count,missing,sum,sumOfSquares,mean,stddev
1,0,391184,1204,0,22011521,1.250949e+12,18281.994,26558.397
2,0,270034,318,0,6608289,4.000626e+11,20780.783,28789.304
3,1354,221758,1249,0,9088713,2.278149e+11,7276.792,11381.997
4,9348,18512,2,0,27860,4.300792e+08,13930.000,6479.927
5,0,887162,39509,0,187617267,6.833412e+12,4748.722,12264.250
6,0,138374,854,0,7817990,1.450356e+11,9154.555,9280.402
7,0,153848,1052,0,10214011,1.993430e+11,9709.136,9762.830


In [12]:
out2 <- cli$mlt(params = list(q='title:"ecology" AND body:"cell"', mlt.fl='title', mlt.mindf=1, mlt.mintf=1, fl='counter_total_all', rows=5))

In [13]:
out

,min,max,count,missing,sum,sumOfSquares,mean,stddev
counter_total_all,0,1322780,47226,0,263688870,1.119526e+13,5583.552916,14348.70574
alm_twitterCount,0,3438,47226,0,304235,8.148536e+07,6.442108,41.03619
journal,min,max,count,missing,sum,sumOfSquares,mean,stddev
1,0,391184,1204,0,22011521,1.250949e+12,18281.994,26558.397
2,0,270034,318,0,6608289,4.000626e+11,20780.783,28789.304
3,1354,221758,1249,0,9088713,2.278149e+11,7276.792,11381.997
4,9348,18512,2,0,27860,4.300792e+08,13930.000,6479.927
5,0,887162,39509,0,187617267,6.833412e+12,4748.722,12264.250
6,0,138374,854,0,7817990,1.450356e+11,9154.555,9280.402
7,0,153848,1052,0,10214011,1.993430e+11,9709.136,9762.830


In [16]:
out <- cli$highlight(params = list(q='alcohol', hl.fl = 'abstract', rows=2))
out

names,abstract
10.1371/journal.pone.0201042,Acute <em>alcohol</em> administration can lead to a loss of control over drinking. Several models argue
10.1371/journal.pone.0185457,Objectives: <em>Alcohol</em>-related morbidity and mortality are significant public health issues


In [36]:
cli$search(params = list(q='_val_:"product(counter_total_all,alm_twitterCount)"',
  rows=5, fl='id,title', fq='doc_type:full'))

id,title
10.1371/journal.pmed.0020124,Why Most Published Research Findings Are False
10.1371/journal.pone.0141854,"A Multi-Level Bayesian Analysis of Racial Bias in Police Shootings at the County-Level in the United States, 2011–2014"
10.1371/journal.pcbi.1005619,Ten simple rules for structuring papers
10.1371/journal.pone.0016885,Long-Term Follow-Up of Transsexual Persons Undergoing Sex Reassignment Surgery: Cohort Study in Sweden
10.1371/journal.pone.0185809,More than 75 percent decline over 27 years in total flying insect biomass in protected areas


In [37]:
cli$search(params = list(q='_val_:"max(counter_total_all)"',
    rows=5, fl='id,counter_total_all', fq='doc_type:full'))

id,counter_total_all
10.1371/journal.pmed.0020124,2728832
10.1371/journal.pcbi.1003149,1322780
10.1371/annotation/80bd7285-9d2d-403a-8e6f-9c375bf977ca,1235195
10.1371/journal.pone.0141854,887162
10.1371/journal.pcbi.0030102,872604


In [38]:
conn <- SolrClient$new(scheme = "https", host = "bison.usgs.gov", path = "solr/occurrences/select", port = NULL)
conn$search(params = list(q = '*:*', fl = c('decimalLatitude','decimalLongitude','scientificName'), rows = 2))


decimalLongitude,scientificName,decimalLatitude
-75.41667,Setophaga coronata coronata,37.91667
-75.41667,Setophaga coronata coronata,37.91667


In [39]:
conn <- SolrClient$new(scheme = "https", host = "bison.usgs.gov", path = "solr/scientificName/select", port = NULL)
conn$search(params = list(q = '*:*'))

scientificName,_version_
Dictyopteris polypodioides,1.565325e+18
Lonicera iberica,1.565325e+18
Epuraea ambigua,1.565325e+18
Pseudopomala brachyptera,1.565325e+18
Didymosphaeria populina,1.565325e+18
Sanoarca,1.565325e+18
Celleporina ventricosa,1.565325e+18
Trigonurus crotchi,1.565325e+18
Ceraticelus laticeps,1.565325e+18
Micraster acutus,1.565325e+18
